# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_ref_data)

extension : .pickle
Loaded 180940 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_ref.pickle


In [3]:
df.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,NaN
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,NaN
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,NaN
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,NaN
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,NaN


# 

# Preprocessing

In [22]:
df_temp = df[['u']]

In [24]:
df_temp.shape

(180940, 1)

In [25]:
df_temp.drop_duplicates(inplace=True)

/tmp/ipykernel_7969/3133127905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop_duplicates(inplace=True)


In [28]:
df_temp.reset_index(inplace=True, drop=True)

In [29]:
df_temp.shape

(9140, 1)

In [30]:
df_temp.head()

,u
0,나 중간고사 반에서 1등했어
1,중간고사 점수 내가 반에서 제일 잘 받음
2,나 반에서 중간 성적 제일 좋아
3,우리 반에서 내가 시험 제일 잘 봤다
4,중간고사 반 1등 먹음


# 

# Dataset

In [32]:
from datasets import load_dataset

ModuleNotFoundError: No module named '_lzma'

In [ ]:
datasets = load_dataset("klue", "sts")

# 

# Train

In [4]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [15]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device: {device}")

device: mps


### logger init

In [5]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

### model

In [6]:
embedding_model = models.Transformer(config.model_name)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [9]:
model_save_path = config.output_path + config.model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

### sentence embedding pooler

In [11]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [20]:
model = SentenceTransformer(modules=[embedding_model, pooler], device=device)

In [21]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
